<a href="https://colab.research.google.com/github/ksmikko/Deep-Trading/blob/master/RI_F60.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорт библиотек

In [ ]:
import pandas as pd # Импорт модуля
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
!pip install pandas-datareader
import pandas_datareader as pdr
import requests
import datetime
import pathlib
# !pip install apimoex
# import apimoex



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Загрузка данных

In [ ]:
from google.colab import files # Импорт данных в коллаб
data = files.upload() # Данные сохраняются в сессионное хранилище

Saving RI_60.txt to RI_60.txt


In [ ]:

# Чтение данных
ri = pd.read_csv("/content/RI_60.txt")
ri.columns=['tiker','per','date','time','open','high','low','close','volume'] # Добавление заголовков столбцов
ri = ri.drop(['time','tiker','per','volume'], axis = 1)
ri['change'] = ri['close'].pct_change() # Изменение за один день
ri['change2'] = ri['close'].pct_change(2) # Изменение за два дня
ri['change3'] = ri['close'].pct_change(3) # Изменение за три дня
ri['change4'] = ri['close'].pct_change(4) # Изменение за четыре дня
ri['change5'] = ri['close'].pct_change(5) # Изменение за пять дней
ri['change6'] = ri['close'].pct_change(6) # Изменение за шесть дней
ri['change7'] = ri['close'].pct_change(7) # Изменение за семь дней
ri['change8'] = ri['close'].pct_change(8) # Изменение за восемь дней
 # Нижний хвост свечи
ri['close-low'] = (ri['close'] - ri['low'])/ri['open']*100
# Верхний хвост свечи
ri['high-close'] = (ri['high'] - ri['close'])/ri['open']*100
ri['doj'] = (ri['close']-ri['open'])/(ri['high']-ri['low'])
# Заменяем значения na на 0
ri.fillna(0, inplace= True)
ri['close_pred'] = ri['close'].shift(-1) # Сдвиг вверх
ri = ri[:-1] # Убираем последнюю строку
# Удаляем восемь верхние строчки и столбец time
ri =ri.drop(labels=[0,1,2,3,4,5,6,7], axis = 0)
ri = ri.drop('date', axis = 1)
y = ri['close_pred'] # Целевая переменная
x = ri.drop(columns = ['close_pred']) # Признаки
X = np.array(x) # Преобразование датафрема признаков в вектор
Y = np.array(y) # Преобразовние столбца целевой переменной в вектор
# Масштабирование данных
mean = X.mean(axis=0)
std = X.std(axis=0)
X -= mean
X /= std
X.shape
from tensorflow.keras.models import load_model # Загрузка модели из сессионного хранилища
model = load_model("/content/model_best_ri_60_val_mae(epoch).h5")
# Истинные и предсказанные значения
pd.DataFrame({
    'true': np.squeeze(Y),
    'pred': np.squeeze(model.predict(X))
}).tail(20)


99/99 [==============================] - 0s 2ms/step


,true,pred
3129,98370.0,99268.750000
3130,98400.0,99274.656250
3131,98440.0,99298.140625
3132,98620.0,99343.679688
3133,98700.0,99481.640625
3134,100010.0,99598.218750
3135,99830.0,100546.695312
3136,99680.0,100598.640625
3137,99270.0,100624.625000
3138,99080.0,100097.335938


In [ ]:
ri

In [ ]:
X.shape

(3138, 15)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
tf.random.set_seed(15)


model = Sequential()
model.add(Dense(45, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(15, activation='relu'))
model.add(Dense(1))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 45)                720       
                                                                 
 dense_1 (Dense)             (None, 15)                690       
                                                                 
 dense_2 (Dense)             (None, 1)                 16        
                                                                 
Total params: 1,426
Trainable params: 1,426
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Сохранение модели
from keras import callbacks
model_checkpoint = callbacks.ModelCheckpoint(
    filepath= 'model_best_ri60_val_mae(epoch).h5',
    monitor= 'val_mae',
    verbose= 1,
    save_best_only= True,
    save_weights_only= False,
    mode= 'auto',
    save_freq= 'epoch'
)
# model.compile(optimizer='adam', loss='mse', metrics=['mae']) # Компиляция сети
# %%time
# num_epochs = 2000

# model.fit(X, Y,
#           epochs=num_epochs, 
#           batch_size=50,
#           validation_split=0.3,
#           callbacks= [model_checkpoint])

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae']) # Компиляция сети
# %%time
# num_epochs = 2000

# model.fit(X, Y,
#           epochs=num_epochs, 
#           batch_size=50,
#           validation_split=0.3,
#           callbacks= [model_checkpoint])

In [ ]:
%%time
num_epochs = 2000

model.fit(X, Y,
          epochs=num_epochs, 
          batch_size=50,
          validation_split=0.3,
          callbacks= [model_checkpoint])

In [ ]:
# from tensorflow.keras.models import load_model # Загрузка модели из сессионного хранилища
# model = load_model("/content/model_best_ri60_val_mae(epoch).h5")
# Истинные и предсказанные значения
pd.DataFrame({
    'true': np.squeeze(Y),
    'pred': np.squeeze(model.predict(X))
}).tail(20)

99/99 [==============================] - 0s 1ms/step


,true,pred
3129,98370.0,99268.750000
3130,98400.0,99274.656250
3131,98440.0,99298.140625
3132,98620.0,99343.679688
3133,98700.0,99481.640625
3134,100010.0,99598.218750
3135,99830.0,100546.695312
3136,99680.0,100598.640625
3137,99270.0,100624.625000
3138,99080.0,100097.335938
